In [2]:
import pandas as pd

top_hmc_genes = pd.read_excel('./top_hmc_genes.xlsx').to_dict()
top_hmc_genes.pop("Unnamed: 0")


{0: 23,
 1: 22,
 2: 17,
 3: 19,
 4: 18,
 5: 13,
 6: 2,
 7: 10,
 8: 9,
 9: 12,
 10: 1,
 11: 16,
 12: 11,
 13: 15,
 14: 20,
 15: 3,
 16: 8,
 17: 21,
 18: 5,
 19: 6,
 20: 14,
 21: 7,
 22: 4,
 23: 0}

In [3]:
import subprocess

genes_info = {}

for gene_name in top_hmc_genes["gene_name"].values():
    gene_info = subprocess.check_output(
        "grep {gene_name} ./feature_references/fig5_features/mm39_RefSeqC_select_merged_modified.bed".format(
        gene_name=gene_name), shell=True).decode(
        "UTF-8").split()
    gene_entry = {
        gene_name : 
        {
        "chromosome" : gene_info[0],
        "chromStart" : gene_info[1],
        "chromEnd" : gene_info[2],
        "strand" : gene_info[5]
        }
        }
    
    genes_info.update(gene_entry)

In [29]:
target_bed = pd.DataFrame(genes_info).T.reset_index()[["chromosome", "chromStart", "chromEnd"]]
target_bed.to_csv('./feature_references/fig7_genes/target_genes.bed', sep="\t", header=None, index=None)

In [5]:
from modbampy import ModBam

with ModBam('./data/prom_R10.4.1_E8.2_WGS_brain_DMR_regions.bam') as bam: 
    gene_list = []
    for gene in genes_info.values():
        read_list = []
        for read in bam.reads(gene["chromosome"], gene["chromStart"], gene["chromEnd"]):
            read_sites = []
            for mod_position in read.mod_sites:
                mod_position_info = [gene["chromosome"], gene["chromStart"], gene["chromEnd"], gene["geneName"]]
                for datum in mod_position:
                    mod_position_info.append(datum)
                read_sites.append(mod_position_info)
            read_list.extend(read_sites)
        gene_list.extend(read_list)
    names=["chromosome", "chromStart", "chromEnd", "geneName", "read", "refPos", "qPos", "refStrand", "modStrand", "base", "mod", "score"]
    gene_df = pd.DataFrame(gene_list, columns=names)

NameError: name 'coord_dict' is not defined